In [1]:
from seleniumwire import webdriver
from selenium.webdriver.common.by import By
import requests,os,sys,json,re,zlib,brotli
from bs4 import BeautifulSoup
from time import sleep
from pprint import pprint
import pandas as pd
from time import sleep

In [2]:
from leagues.models import Season,Competition
from base_app.decorators import catch_iteration_errors
from base_app.helpers import fetch_configurations
from base_app.models import ConfigItems

In [3]:
CONFIG = fetch_configurations()

In [4]:
all_comps_sa = Competition.objects.filter(event_data_available=True)

In [5]:
all_comps_sa[0].name_scoresaway

'AFC Champions League Elite'

## Navigate to page, get all values and links in the select box. SIMPLE

In [6]:
def is_recent(entry, threshold=int(CONFIG.get('START_YEAR'))):
    try:
        # Split by '/' or just wrap in a list if single year
        years = [int(y) for y in str(entry).split('/') if y.isdigit()]
        # Keep if any year is >= threshold
        return any(y >= threshold for y in years)
    except ValueError:
        return False

In [46]:
@catch_iteration_errors
def fetch_event_season_urls(index,comp):
    driver = webdriver.Chrome()
    driver.get(comp.event_data_url)
    sleep(10)
    try:
        select_box = driver.find_element(By.CSS_SELECTOR, "#season-select")
    except:
        print("error aagya tha")
        driver.refresh()
        sleep(10)
        select_box = driver.find_element(By.CSS_SELECTOR, "#season-select")
    
    links = {season.get_attribute('innerText') : f"https://www.scoresway.com{season.get_property('value')}" for season in select_box.find_elements(By.CSS_SELECTOR, "option")}
    # print(links)
    driver.quit()
    return links

for i,comp in enumerate(all_comps_sa[4:],5) :
    link_dicts = fetch_event_season_urls(i,comp)
    exisiting_seasons = Season.objects.filter(competition = comp).values_list('name',flat=True)
    if comp.name_scoresaway is None :
        print("nahi hua : ",comp.__dict__)
        print()
        continue
    seasons_considered = {yr:value for yr,value in link_dicts.items() if is_recent(yr) and yr not in exisiting_seasons}
    for season_name, event_url in seasons_considered.items():
        Season.objects.create(competition=comp,name=season_name,season_event_url=event_url)
    print(i,comp.competition_name,seasons_considered)


5 J1 League {}
6 Qatar Stars League {}
7 Saudi Pro League {}
8 Thai League 1 {}
9 CAF Champions League {}
10 CAF Confederation Cup {}
11 Egyptian Premier League {}
12 PSL {}
13 Copa Libertadores {}
14 Copa Sudamericana {}
15 Liga Profesional {}
16 Brasileiro Serie A {}
error aagya tha
17 Brasileiro Serie B {}
18 Chilean Primera Division {}
19 Categoria Primera A {}
20 Liga Pro {}
21 Division Profesional {}
error aagya tha
22 Liga 1 {}
23 Primera Division {}
24 CONCACAF Champions Cup {}
error aagya tha
25 Canadian Premier League {}
26 Liga MX {}
27 Major League Soccer {}
28 UEFA Champions League {}
29 UEFA Europa League {}
error aagya tha
30 UEFA Conference League {}
31 Austrian Bundesliga {}
32 Jupiler Pro League {}
error aagya tha
33 Challenger Pro League {}
34 HNL {}
35 Czech First League {}
error aagya tha
36 Danish Superliga {}
37 Danish 1. Division {}
38 Premier League {}
error aagya tha
39 EFL Championship {}
40 EFL League One {}
41 EFL League Two {}
42 Veikkausliiga {}
43 Ligue 

## FOR FOTMOB IT"LL WORK SAME

In [7]:
all_comps_fm = Competition.objects.filter(shot_data_available=True)

In [8]:
all_comps_fm

<QuerySet [<Competition: A-League - Australia>, <Competition: Indian Super League - India>, <Competition: J1 League - Japan>, <Competition: Saudi Pro League - Saudi Arabia>, <Competition: Copa Libertadores - CONMEBOL>, <Competition: Liga Profesional - Argentina>, <Competition: Brasileiro Serie A - Brazil>, <Competition: Liga MX - Mexico>, <Competition: Major League Soccer - USA>, <Competition: UEFA Champions League - UEFA>, <Competition: UEFA Europa League - UEFA>, <Competition: UEFA Conference League - UEFA>, <Competition: Austrian Bundesliga - Austria>, <Competition: Jupiler Pro League - Belgium>, <Competition: Danish Superliga - Denmark>, <Competition: Premier League - England>, <Competition: EFL Championship - England>, <Competition: EFL League One - England>, <Competition: EFL League Two - England>, <Competition: Ligue 1 - France>, '...(remaining elements truncated)...']>

In [11]:
@catch_iteration_errors
def fetch_shot_season_urls(index,comp):
    driver = webdriver.Chrome()
    driver.get(comp.shot_data_url)
    sleep(10)
    body = {}
    for request in driver.requests:
        if request.response and "https://www.fotmob.com/api/data/leagues" in request.url  :
            encoding = request.response.headers.get('Content-Encoding', '')
            raw_body = request.response.body
            if encoding == 'br':
                body = brotli.decompress(raw_body).decode('utf-8')
            elif encoding == 'gzip':
                body = zlib.decompress(raw_body, zlib.MAX_WBITS | 16).decode('utf-8')
            elif encoding == 'deflate':
                body = zlib.decompress(raw_body).decode('utf-8')
            else:
                body = raw_body.decode('utf-8')
    driver.quit()
    with open("Example.txt","w",encoding="utf-8") as f :
        f.write(str(body))
    exisiting_seasons = Season.objects.filter(competition = comp).exclude(season_shot_url = "").values_list('name',flat=True)
    fm_seasons = json.loads(body).get('allAvailableSeasons')
    all_seasons = [x for x in fm_seasons if is_recent(x) and x not in exisiting_seasons ]
    all_season_urls = {}
    for season in all_seasons:
        if season not in all_season_urls:
            try :
                current_obj = Season.objects.get(competition=comp,name=season)
            except Exception as e :
                print(comp,season)
                raise(e)
            link = f'{comp.shot_data_url}?season={season.replace(r"/","-").replace(" ","+")}'
            current_obj.season_shot_url = link
            current_obj.save()
    

err = []
for i,comp in enumerate(all_comps_fm,1) :
    print(f"Working on {comp.competition_name}")
    try : 
        link_dicts = fetch_shot_season_urls(i,comp)
    except :
        err.append(comp)


Working on A-League
❌ Error in iteration 1: the JSON object must be str, bytes or bytearray, not dict
 ❌ Line number: 339
 ❌Error line: raise TypeError(f'the JSON object must be str, bytes or bytearray, '
    Closing any open connections...
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
Working on Indian Super League
Indian Super League - India 2025/2026
❌ Error in iteration 2: Season matching query does not exist.
 ❌ Line number: 633
 ❌Error line: raise self.model.DoesNotExist(
    Closing any open connections...
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
Working on J1 League
J1 League - Japan 2026
❌ Error in iteration 3: Season matching query does not exist.
 ❌ Line number: 633
 ❌Error line: raise self.model.DoesNotExist(
    Closing any open connections...
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
Working on Saudi Pro League
Working on Copa Libertadores
Working on Liga Profesional
Working on Brasileiro Serie A
Working on Liga MX
Liga MX - Mexico 2025/2026 - Clausura
❌ Error in iteration 8: Season matching query does n

In [21]:
err_items = [all_comps_fm[0],all_comps_fm[2],all_comps_fm[1],all_comps_fm[7]]
all_comps_fm[7]

<Competition: Liga MX - Mexico>

In [51]:
def fetch_shot_season_urls_internal(index,comp):
    driver = webdriver.Chrome()
    driver.get(comp.shot_data_url)
    sleep(10)
    body = {}
    for request in driver.requests:
        if request.response and "https://www.fotmob.com/api/data/leagues" in request.url  :
            encoding = request.response.headers.get('Content-Encoding', '')
            raw_body = request.response.body
            if encoding == 'br':
                body = brotli.decompress(raw_body).decode('utf-8')
            elif encoding == 'gzip':
                body = zlib.decompress(raw_body, zlib.MAX_WBITS | 16).decode('utf-8')
            elif encoding == 'deflate':
                body = zlib.decompress(raw_body).decode('utf-8')
            else:
                body = raw_body.decode('utf-8')
    driver.quit()
    with open("Example.txt","w",encoding="utf-8") as f :
        f.write(str(body))
    exisiting_seasons = Season.objects.filter(competition = comp).exclude(season_shot_url = "").values_list('name',flat=True)
    fm_seasons = json.loads(body).get('allAvailableSeasons')
    all_seasons = [x for x in fm_seasons if is_recent(x) and x not in exisiting_seasons ]
    all_season_urls = {}
    for season in all_seasons:
        if season not in all_season_urls:
            try :
                current_obj = Season.objects.get(competition=comp,name_fotmob=season)
            except Exception as e :
                print(comp,season)
                raise(e)
            link = f'{comp.shot_data_url}?season={season.replace(r"/","-").replace(" ","+")}'
            current_obj.season_shot_url = link
            current_obj.save()
   

In [61]:
fetch_shot_season_urls_internal(0,err_items[2])
# err_items[1].id

In [63]:
print(err_items[3],'2025/2026')
Season.objects.get(competition=err_items[3],name='2025/2026')


Liga MX - Mexico 2025/2026


MultipleObjectsReturned: get() returned more than one Season -- it returned 2!

### 

In [22]:
jap = next(x for x in all_comps_fm if x.name_fotmob=='J. League')
mex = next(x for x in all_comps_fm if x.name_fotmob=='Liga MX')

In [23]:
driver = webdriver.Chrome()
driver.get(mex.shot_data_url)

for request in driver.requests:
    if request.response and "https://www.fotmob.com/api/data/leagues" in request.url:
        encoding = request.response.headers.get('Content-Encoding', '')
        raw_body = request.response.body
        print(encoding)

        if encoding == 'br':
            body = brotli.decompress(raw_body).decode('utf-8')
        elif encoding == 'gzip':
            body = zlib.decompress(raw_body, zlib.MAX_WBITS | 16).decode('utf-8')
        elif encoding == 'deflate':
            body = zlib.decompress(raw_body).decode('utf-8')
        else:
            body = raw_body.decode('utf-8')
        print(body)
driver.quit()
all_seasons = [x for x in json.loads(body).get('allAvailableSeasons') if is_recent(x)]
# all_season_urls = {}

br
{"tabs":["overview","table","fixtures","stats","transfers","seasons"],"allAvailableSeasons":["2025/2026 - Clausura","2025/2026 - Apertura","2024/2025 - Clausura","2024/2025 - Apertura","2023/2024 - Clausura","2023/2024 - Apertura","2022/2023 - Clausura","2022/2023 - Apertura","2021/2022 - Clausura","2021/2022 - Apertura","2020/2021 - Clausura","2020/2021 - Apertura","2019/2020 - Clausura","2019/2020 - Apertura","2018/2019 - Clausura","2018/2019 - Apertura","2017/2018 - Clausura","2017/2018 - Apertura","2016/2017 - Clausura","2016/2017 - Apertura","2015/2016 - Clausura","2015/2016 - Apertura","2014/2015 - Clausura","2014/2015 - Apertura","2013/2014 - Clausura","2013/2014 - Apertura","2012/2013 - Clausura","2012/2013 - Apertura","2011/2012 - Clausura","2011/2012 - Apertura","2010/2011 - Clausura","2010/2011 - Apertura"],"details":{"id":230,"type":"league","name":"Liga MX","selectedSeason":"2025/2026 - Clausura","latestSeason":"2025/2026 - Clausura","shortName":"Liga MX","country":"MEX

In [24]:
[x for x in json.loads(body).get('allAvailableSeasons') if is_recent(x)][:2]

['2025/2026 - Clausura', '2025/2026 - Apertura']

In [25]:
season_links = {}
for x in all_seasons[:2]:
    season_type = x.split('-')[-1].strip()
    if season_type not in season_links :
        season_links[season_type] = []
    season_links[season_type].append((x,f'{mex.shot_data_url}?season={x.replace(r"/","-").replace(" ","+")}'))

In [26]:
pprint(season_links)

{'Apertura': [('2025/2026 - Apertura',
               'https://www.fotmob.com/leagues/230/overview/liga-mx?season=2025-2026+-+Apertura')],
 'Clausura': [('2025/2026 - Clausura',
               'https://www.fotmob.com/leagues/230/overview/liga-mx?season=2025-2026+-+Clausura')]}


In [27]:
mexico_items = Season.objects.filter(competition__id=27,name__contains='26')

In [28]:
mexico_items[0].__dict__

{'_state': <django.db.models.base.ModelState at 0x203bc281d90>,
 'id': 594,
 'competition_id': 27,
 'name': '2025/2026',
 'name_fotmob': '2025/2026 - Apertura',
 'season_event_url': 'https://www.scoresway.com/en_GB/soccer/liga-mx-2025-2026/aegyls91smdw9kipjgbsu8tn8/fixtures',
 'season_shot_url': 'https://www.fotmob.com/leagues/230/overview/liga-mx?season=2025-2026+-+Apertura'}

In [29]:
for i,item in enumerate(mexico_items):
    name,link = season_links['Clausura'][i]
    # name,link = season_links['Clausura'][i]
    print(f"""
            competition = {item.competition}
            name = {item.name}
            name_fotmob = {name}
            season_event_url = {item.season_event_url}
            season_shot_url = {link}
            id = {item.id}
            """)
    Season.objects.create(
            competition = item.competition,
            name = item.name,
            name_fotmob = name,
            season_event_url = item.season_event_url,
            season_shot_url = link)

           # item.save())
    # item.name_fotmob = f"{item.name} - Apertura"
    # item.season_shot_url = link
    
    # item.save()
    print(item.__dict__)
    # break


            competition = Liga MX - Mexico
            name = 2025/2026
            name_fotmob = 2025/2026 - Clausura
            season_event_url = https://www.scoresway.com/en_GB/soccer/liga-mx-2025-2026/aegyls91smdw9kipjgbsu8tn8/fixtures
            season_shot_url = https://www.fotmob.com/leagues/230/overview/liga-mx?season=2025-2026+-+Clausura
            id = 594
            


IntegrityError: UNIQUE constraint failed: leagues_season.competition_id, leagues_season.name, leagues_season.name_fotmob